In [2]:
#os.environ['THEANO_FLAGS'] = "device=gpu1"  
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32,lib.cnmem=1"
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"
import os    
#%pylab inline
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score


import theano
from keras.layers import Input, Dense
from keras.layers import Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.utils.np_utils import *

from sklearn.cross_validation import train_test_split

In [2]:
x_train = np.genfromtxt('x_train.out')
y_train = np.genfromtxt('y_train.out')
vx_train = np.genfromtxt('vx_train.out')
vy_train = np.genfromtxt('vy_train.out')
x_test = np.genfromtxt('x_test.out')
y_test = np.genfromtxt('y_test.out')

In [3]:
print (x_train.shape)
print (y_train.shape)
print (vx_train.shape)
print (vy_train.shape)
print (x_test.shape)
print (y_test.shape)

(50000, 784)
(50000,)
(10000, 784)
(10000,)
(10000, 784)
(10000,)


In [4]:
label_train=to_categorical(y_train)
label_valid=to_categorical(vy_train)
label_test= to_categorical(y_test)

In [5]:
print (x_train.shape)
print (label_train.shape)
print (vx_train.shape)
print (label_valid.shape)
print (x_test.shape)
print (label_test.shape)

(50000, 784)
(50000, 10)
(10000, 784)
(10000, 10)
(10000, 784)
(10000, 10)


In [6]:
# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

In [17]:
# define vars
input_num_units = 784
#hidden_num_units = 500

hidden1_num_units = 500
hidden2_num_units = 500
hidden3_num_units = 500
hidden4_num_units = 500
hidden5_num_units = 500

output_num_units = 10

epochs = 25
batch_size = 128

In [27]:
# Shallow Network
model = Sequential([
  Dense(output_dim=hidden_num_units, input_dim=input_num_units, activation='relu'),
  Dense(output_dim=output_num_units, input_dim=hidden_num_units, activation='softmax'),
])

In [31]:
# Deep network
model = Sequential([
 Dense(output_dim=hidden1_num_units, input_dim=input_num_units, activation='relu'),
 Dense(output_dim=hidden2_num_units, input_dim=hidden1_num_units, activation='relu'),
 Dense(output_dim=hidden3_num_units, input_dim=hidden2_num_units, activation='relu'),
 Dense(output_dim=hidden4_num_units, input_dim=hidden3_num_units, activation='relu'),
 Dense(output_dim=hidden5_num_units, input_dim=hidden4_num_units, activation='relu'),

Dense(output_dim=output_num_units, input_dim=hidden5_num_units, activation='softmax'),
 ])

In [18]:
dropout_ratio = 0.2

model = Sequential([
 Dense(output_dim=hidden1_num_units, input_dim=input_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(output_dim=hidden2_num_units, input_dim=hidden1_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(output_dim=hidden3_num_units, input_dim=hidden2_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(output_dim=hidden4_num_units, input_dim=hidden3_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(output_dim=hidden5_num_units, input_dim=hidden4_num_units, activation='relu'),
 Dropout(dropout_ratio),

Dense(output_dim=output_num_units, input_dim=hidden5_num_units, activation='softmax'),
 ])

In [19]:
# compile the model with necessary attributes
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
trained_model = model.fit(
    x_train, label_train, nb_epoch=epochs, batch_size=batch_size, 
    validation_data=(vx_train, label_valid))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 9s - loss: 0.3218 - acc: 0.9003 - val_loss: 0.1265 - val_acc: 0.9627
Epoch 2/25
50000/50000 [==============================] - 9s - loss: 0.1404 - acc: 0.9586 - val_loss: 0.1089 - val_acc: 0.9682
Epoch 3/25
50000/50000 [==============================] - 9s - loss: 0.1009 - acc: 0.9710 - val_loss: 0.1038 - val_acc: 0.9712
Epoch 4/25
50000/50000 [==============================] - 10s - loss: 0.0881 - acc: 0.9743 - val_loss: 0.1069 - val_acc: 0.9725
Epoch 5/25
50000/50000 [==============================] - 9s - loss: 0.0773 - acc: 0.9778 - val_loss: 0.0834 - val_acc: 0.9771
Epoch 6/25
50000/50000 [==============================] - 10s - loss: 0.0629 - acc: 0.9819 - val_loss: 0.0853 - val_acc: 0.9762
Epoch 7/25
50000/50000 [==============================] - 9s - loss: 0.0599 - acc: 0.9822 - val_loss: 0.0854 - val_acc: 0.9771
Epoch 8/25
50000/50000 [==============================] - 1

In [7]:
# reshape data

#train_x_temp = x_train.reshape(-1, 28, 28, 1)
#val_x_temp = vx_train.reshape(-1, 28, 28, 1)
#test_x_temp=x_test.reshape(-1, 28, 28, 1)

train_x_temp = x_train.reshape(-1,1, 28, 28)
val_x_temp = vx_train.reshape(-1,1, 28, 28)
test_x_temp=x_test.reshape(-1,1, 28, 28)
print(train_x_temp.shape)
print(val_x_temp.shape)
print(test_x_temp.shape)


# define vars
input_shape = (784,)
input_reshape = (1,28, 28)

conv_num_filters = 5
conv_filter_size = 5

pool_size = (2, 2)

hidden_num_units = 50
output_num_units = 10

epochs = 5
batch_size = 128

model = Sequential([
 #InputLayer(input_shape=input_reshape),

 Convolution2D(25, 5, 5, border_mode='same',activation='relu',
              input_shape=input_reshape),
 MaxPooling2D(pool_size=pool_size),

 Convolution2D(25, 5, 5, border_mode='same',activation='relu'),
 MaxPooling2D(pool_size=pool_size),

 Convolution2D(25, 4, 4, border_mode='same',activation='relu'),

 Flatten(),

 Dense(output_dim=hidden_num_units, activation='relu'),

 Dense(output_dim=output_num_units, input_dim=hidden_num_units, activation='softmax'),
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

trained_model_conv = model.fit(train_x_temp, label_train, 
                               nb_epoch=epochs, batch_size=batch_size, 
                               validation_data=(val_x_temp, label_valid))

(50000, 1, 28, 28)
(10000, 1, 28, 28)
(10000, 1, 28, 28)
Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 50s - loss: 0.2616 - acc: 0.9222 - val_loss: 0.1017 - val_acc: 0.9668
Epoch 2/5
50000/50000 [==============================] - 50s - loss: 0.0681 - acc: 0.9786 - val_loss: 0.0635 - val_acc: 0.9810
Epoch 3/5
50000/50000 [==============================] - 50s - loss: 0.0470 - acc: 0.9851 - val_loss: 0.0434 - val_acc: 0.9874
Epoch 4/5
50000/50000 [==============================] - 51s - loss: 0.0376 - acc: 0.9879 - val_loss: 0.0424 - val_acc: 0.9870
Epoch 5/5
50000/50000 [==============================] - 52s - loss: 0.0312 - acc: 0.9901 - val_loss: 0.0592 - val_acc: 0.9811


In [41]:
#pred = model.predict_classes(x_test)
pred = model.predict_classes(test_x_temp)

 9984/10000 [============================>.] - ETA: 0s

In [42]:
pred

array([7, 2, 1, ..., 4, 5, 6])

In [43]:
pred= to_categorical(pred)

In [44]:
pred

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [45]:
label_test

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [46]:
num=len(label_test)
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(pred[i],label_test[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "digits")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")

tested  10000 digits
correct:  9877 wrong:  123 error rate:  1.23 %
got correctly  98.77 %
